<a href="https://colab.research.google.com/github/manjeshsinghh/NLP-project/blob/main/NLP_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install transformers torch nltk rouge


In [ ]:
# Load the dataset
import pandas as pd
data_path = '/content/amazon.csv.zip'
df = pd.read_csv(data_path)

In [ ]:
import pandas as pd
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import torch
from nltk.translate.bleu_score import sentence_bleu
from rouge import Rouge

# Initialize the GPT-2 model and tokenizer
model_name = "gpt2"  # Using the base GPT-2 model
tokenizer = GPT2Tokenizer.from_pretrained(model_name)
model = GPT2LMHeadModel.from_pretrained(model_name)
device = "cuda" if torch.cuda.is_available() else "cpu"
model.to(device)

# Function to generate text with sampling
def generate_text(prompt, max_length=150):
    inputs = tokenizer.encode(prompt, return_tensors="pt").to(device)

    # Ensure the length of input is not exceeding the max_length
    input_length = inputs.shape[1]
    max_input_length = 1024  # GPT-2's maximum token length

    if input_length > max_input_length:
        inputs = inputs[:, -max_input_length:]

    outputs = model.generate(
        inputs,
        max_new_tokens=max_length,  # Use max_new_tokens to control number of generated tokens
        num_return_sequences=1,
        do_sample=True,  # Enable sampling
        temperature=0.7,
        top_k=50,
        top_p=0.95,
        pad_token_id=tokenizer.eos_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)

# Reward function using BLEU and ROUGE
def reward_function(generated_text, reference_text):
    bleu_score = sentence_bleu([reference_text.split()], generated_text.split())

    rouge = Rouge()
    rouge_scores = rouge.get_scores(generated_text, reference_text)
    rouge_l_score = rouge_scores[0]['rouge-l']['f']

    # Combine BLEU and ROUGE-L scores (adjust weights as needed)
    combined_score = 0.5 * bleu_score + 0.5 * rouge_l_score

    return combined_score

# Iterative feedback loop with manual feedback input
def iterative_feedback(prompt, reference_text, iterations=5):
    for i in range(iterations):
        # Generate text
        generated_text = generate_text(prompt)
        print(f"Iteration {i + 1}:")
        print(f"Generated Text:\n{generated_text}\n")

        # Evaluate feedback using automatic metrics (BLEU and ROUGE)
        reward = reward_function(generated_text, reference_text)
        print(f"Automatic Reward (BLEU + ROUGE-L): {reward}")

        # Request manual feedback from the user
        manual_score = float(input("Rate the generated text on a scale of 1 to 10 (higher is better): "))
        print(f"Manual Feedback Score: {manual_score}\n")

        # Combine the automatic reward and manual feedback
        combined_reward = (0.7 * manual_score / 10) + (0.3 * reward)  # Adjust the weights as needed
        print(f"Combined Reward (Manual + Automatic): {combined_reward}\n")

        # Simulate updating model based on feedback (in practice, you'd retrain or fine-tune the model)
        print("Adjusting model based on feedback (placeholder)\n")
        print("-" * 50)

# Example usage (using review content as a pseudo-reference)

for index, row in df.iterrows():
    product_name = row['product_name']
    description = row['about_product']
    reference_text = row['review_content']  # Using review as a pseudo-reference for demonstration
    prompt = f"Product Name: {product_name}\nDescription: {description}\nGenerate a compelling product description:"

    print(f"\nProcessing product: {product_name}\n")
    iterative_feedback(prompt, reference_text)
    # Break after the first product for brevity, remove if processing all products is required
    break


/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.



Processing product: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)



/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_

Iteration 1:
Generated Text:
Product Name: Wayona Nylon Braided USB to Lightning Fast Charging and Data Sync Cable Compatible for iPhone 13, 12,11, X, 8, 7, 6, 5, iPad Air, Pro, Mini (3 FT Pack of 1, Grey)
Description: High Compatibility : Compatible With iPhone 12, 11, X/XsMax/Xr,iPhone 8/8 Plus,iPhone 7/7 Plus,iPhone 6s/6s Plus,iPhone 6/6 Plus,iPhone 5/5s/5c/se,iPad Pro,iPad Air 1/2,iPad mini 1/2/3,iPod nano7,iPod touch and more apple devices.|Fast Charge&Data Sync : It can charge and sync simultaneously at a rapid speed, Compatible with any charging adaptor, multi-port charging station or power bank.|Durability : Durable nylon braided design with premium aluminum housing and toughened nylon fiber wound tightly around the cord lending it superior durability and adding a bit to its flexibility.|High Security Level : It is designed to fully protect your device from damaging excessive current.Copper core thick+Multilayer shielding, Anti-interference, Protective circuit equipment.|WARRAN